In [ ]:
%pylab inline --no-import-all

import pandas as pd
import seaborn as sns

pd.options.display.max_columns=50

## Load data

In [ ]:
# Data is assumed to be in directory data/
with open('data/colnames.py', 'r') as myfile:
    columns_names_dict = eval(myfile.read())
data_train = pd.read_csv("data/train_ver2.csv", parse_dates=[0,6], skipinitialspace=True, nrows=1000000)
data_train.rename(columns=columns_names_dict, inplace=True)

data_train.head()

# Data Visualization

In [ ]:
data_train.describe()

Looks like we've got outilers in SENIORITY_MNTH

In [ ]:
data_train.SENIORITY_MNTH[data_train.SENIORITY_MNTH < 0] = np.NaN

## Age histogram

In [ ]:
data_train.CUST_AGE.plot.hist(bins = len(data_train.CUST_AGE.unique()))

## Income boxplots

In [ ]:
plt.figure()
fig = sns.boxplot(x="CUSTIND_SEGMNT", y="CUST_HOUSINCOM", data = data_train)
fig.set_yscale('log')

In [ ]:
plt.figure()
fig = sns.boxplot(x="EMPLYMT_STATUS", y="CUST_HOUSINCOM", data = data_train)
fig.set_yscale('log')

## Seniority

In [ ]:
sns.boxplot(x=data_train.SENIORITY_MNTH)